In [57]:
from os import path
from itertools import product
from IPython.display import display
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [58]:
BASE_PATH = '/Users/deepudilip/ML/Coursera/HowToWinDSCompeition/KagglePredictFutureSales'
DATA_PATH = path.join(BASE_PATH, 'data/')

In [59]:
item_cat_df = pd.read_csv(path.join(DATA_PATH, 'item_categories.csv'))
item_df = pd.read_csv(path.join(DATA_PATH, 'items.csv'))
sales_train_df = pd.read_csv(path.join(DATA_PATH, 'sales_train.csv'))
shops_df = pd.read_csv(path.join(DATA_PATH, 'shops.csv'))

In [60]:
test_df = pd.read_csv(path.join(DATA_PATH, 'test.csv'))

# Train data preparation - based on an existing Kaggle kernel

The steps below are inspired from a kaggle kernel [https://www.kaggle.com/dlarionov/feature-engineering-xgboost]
The idea is to create a new bigger training data frame, having one row for each unique shop ID - item ID combination in a given month, with the total sales as zero, in case such a combination is not present in the originally provided training data. At first glance, I am not sure if this is indeed the best approach. My assumption regarding the thought process of the author of this kernel is as follows:
1) Presence of a shop_id and item_id anywhere in the month indicates the existence of such a product and shop in the market, and hence restricting the list of shops / items to those within a month while generating synthetic rows. Assumption behind creation of such a combination is that the shop did not sell that item at all that month. This is a rather vague assumption, but as the author of the kernal claims, this makes the training data more 'similar' to the test data, since in the test data, we have rows for each shop id and each item id

As a first step, this approach is tested. Later, I will try alternative approaches of preparing training data

In [61]:
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train_df[sales_train_df.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique()))))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [62]:
matrix.shape

(10913850, 3)

Now we will combine the merge the above dataframe with the monthly aggregated sales dataframe and fill the 'NA' values for target (which will occur for shop-id - item-id combinations that does not exist in the training dataframe) with zeros.

In [63]:
sales_train_df['revenue'] = sales_train_df['item_cnt_day'] * sales_train_df['item_price']

In [64]:
sales_train_monthly = sales_train_df.groupby(['date_block_num',
                                              'shop_id', 
                                              'item_id']).agg({'item_cnt_day': 'sum'})
sales_train_monthly.columns = ['item_cnt_month']
sales_train_monthly.reset_index(inplace=True)

In [65]:
sales_train_monthly.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [66]:
matrix = (matrix.merge(sales_train_monthly,
                     on=['date_block_num',
                         'shop_id',
                         'item_id'],
                     how='left')
         .fillna(0))

In [67]:
matrix.shape

(10913850, 4)

In [90]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0


It is mentioned in the problem statement that the target column is clipped between 0,20. This is done below

In [68]:
matrix['item_cnt_month'] = matrix['item_cnt_month'].clip(0, 20)

Preproces the dataframe for shops based and items / item categories and then merge with the above dataframe

In [69]:
# Shops dataframe
shops_df.loc[shops_df['shop_name'] == 57, 'shop_name'] = 0
shops_df.loc[shops_df['shop_name'] == 58, 'shop_name'] = 1
shops_df.loc[shops_df['shop_name'] == 10, 'shop_name'] = 11
shops_df['city'] = shops_df['shop_name'].apply(lambda x: x.split()[0])
shops_df.loc[shops_df['city'] == '!Якутск', 'city'] = 'Якутск'
label_encoder = LabelEncoder()
shops_df['city_id'] = label_encoder.fit_transform(shops_df['city'])
shops_df = shops_df[['shop_id', 'city_id']]

# Item categories dataframe
item_cat_df['split'] = item_cat_df['item_category_name'].str.split('-')
item_cat_df['item_type'] = item_cat_df['split'].apply(lambda x: x[0].strip())
item_cat_df['item_sub_type'] = item_cat_df['split'].apply(lambda x: x[1].strip() if len(x) > 1 else x[0])
item_cat_df['item_type_id'] = LabelEncoder().fit_transform(item_cat_df['item_type'])
item_cat_df['item_sub_type_id'] = LabelEncoder().fit_transform(item_cat_df['item_sub_type'])
item_cat_df = item_cat_df[['item_category_id', 'item_type_id', 'item_sub_type_id']]

In [70]:
display(shops_df.head())
display(item_df.head())
display(item_cat_df.head())

,shop_id,city_id
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,item_category_id,item_type_id,item_sub_type_id
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


In [71]:
item_df.drop(['item_name'], axis=1, inplace=True)

In [155]:
np.float16

numpy.float16

In [72]:
test_df = test_df.merge(shops_df, on='shop_id', how='left')
test_df = test_df.merge(item_df, on='item_id', how='left')
test_df = test_df.merge(item_cat_df, on='item_category_id', how='left')

In [73]:
train_df = matrix.copy(deep=True)

In [74]:
train_df = train_df.merge(shops_df, on='shop_id', how='left')
train_df = train_df.merge(item_df, on='item_id', how='left')
train_df = train_df.merge(item_cat_df, on='item_category_id', how='left')

In [75]:
id_columns = ['shop_id', 
              'item_id', 
              'city_id', 
              'item_category_id', 
              'item_type_id',
              '']
for col in id_columns:
    train_df[col] = train_df[col].astype(np.int16)
    test_df[col] = test_df[col].astype(np.int16)

Following are several helper functions to be used for preparing the training and test dataframe

In [76]:
def create_lagged_features(df, lags, col):
    """
    Convenience function to create lagged features (based on date_block_num).
    :param df: A dataframe on which the lagged features have to be created.
    :type df: pandas.DataFrame
    :param lags: A list of lag values for which features need to be created.
    :type lags: List
    :param col: The colum for which the lagged feature is required.
    :type col: str
    :return:
    """
    for lag in lags:
        print("now creating features for lag: ", lag)
        df_shifted = df.loc[:, ['date_block_num', 'shop_id', 'item_id', col]]
        df_shifted['date_block_num'] += lag
        df_shifted.columns = ['date_block_num', 'shop_id', 'item_id', col + '_lagged_' + str(lag)]
        print('merging with original dataframe')
        df = df.merge(df_shifted, on=['date_block_num',
                                      'shop_id',
                                      'item_id'],
                      how='left')
    return df

def get_mean_encoded_features(df, group_cols, agg_col='item_cnt_month', agg_col_alias=None):
    """
    Convenience function to create a mean encoded feature based on an input set of grouping columns.
    :param df:
    :param group_cols:
    :param agg_col:
    :return:
    """
    agg_df = df.groupby(group_cols).agg({agg_col: 'mean'})
    if not agg_col_alias:
        agg_col_alias = 'mean_' + agg_col

    agg_df.columns = [agg_col_alias]
    agg_df.reset_index(inplace=True)

    df = df.merge(agg_df, on=group_cols, how='left')

    return df

def create_lagged_mean_encoded_features(df, group_cols, agg_col, lags, agg_col_alias=None):
    """
    Convenience function to create a get a set of lagged mean encoded feature based on an input set of grouping columns
    and lags.
    :todo add docu
    :param df:
    :param group_cols:
    :param agg_col:
    :param agg_col_alias:
    :return:
    """
    if not agg_col_alias:
        agg_col_alias = 'mean_' + agg_col
    df = get_mean_encoded_features(df, group_cols, agg_col, agg_col_alias)
    df = create_lagged_features(df, lags, agg_col_alias)

    return df

In [144]:
matrix.columns, shops_df.columns, item_df.columns, item_cat_df.columns

(Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month'], dtype='object'),
 Index(['shop_id', 'city_id'], dtype='object'),
 Index(['item_id', 'item_category_id'], dtype='object'),
 Index(['item_category_id', 'item_type_id', 'item_sub_type_id'], dtype='object'))

There are several possible combinations of the above columns that can be used to creating mean encoded features. Also, for each such combination, several lags (1,2,3,6,12) can be created. These have to be experimented. It is not clear why the Kaggle kernel has chosen the following set of combinations. We will use this as a first step and later try out new features

In [77]:
groups_for_mean_encode = [
                          ['date_block_num'],
                          ['date_block_num', 'item_id'],
                          ['date_block_num', 'shop_id'],
                          ['date_block_num', 'item_category_id'],
                          ['date_block_num', 'shop_id', 'item_category_id'],
                          ['date_block_num', 'shop_id', 'item_type_id'],
                          ['date_block_num', 'shop_id', 'item_sub_type_id'],
                          ['date_block_num', 'city_id'],
                          ['date_block_num', 'item_id', 'city_id'],
                          ['date_block_num', 'item_type_id'],
                          ['date_block_num', 'item_sub_type_id']
                         ]
lags_for_mean_encode = [
                        [1],
                        [1, 2, 3, 6, 12],
                        [1, 2, 3, 6, 12],
                        [1],
                        [1],
                        [1],
                        [1],
                        [1],
                        [1],
                        [1],
                        [1]
                       ]

In [79]:
train_df.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_id,item_category_id,item_type_id,item_sub_type_id
0,0,0,19,0.0,29,40,11,4
1,0,0,27,0.0,29,19,5,10
2,0,0,28,0.0,29,30,8,55
3,0,0,29,0.0,29,23,5,16
4,0,0,32,6.0,29,40,11,4


In [80]:
for group, lags in zip(groups_for_mean_encode, lags_for_mean_encode):
    agg_col_alias = '_'.join(['_'.join([item for item in col.split('_')[0:-1]]) for col in group])
    agg_col_alias = re.sub('_block', '', agg_col_alias) + '_avg_item_cnt'
    print('adding mean encoded feature for ' + agg_col_alias + ' with lags ', ', '.join([str(lag) for lag in lags]))
    train_df = create_lagged_mean_encoded_features(train_df, group, 'item_cnt_month', lags, agg_col_alias)

adding mean encoded feature for date_avg_item_cnt with lags  1
now creating features for lag:  1
merging with original dataframe
adding mean encoded feature for date_item_avg_item_cnt with lags  1, 2, 3, 6, 12
now creating features for lag:  1
merging with original dataframe
now creating features for lag:  2
merging with original dataframe
now creating features for lag:  3
merging with original dataframe
now creating features for lag:  6
merging with original dataframe
now creating features for lag:  12
merging with original dataframe
adding mean encoded feature for date_shop_avg_item_cnt with lags  1, 2, 3, 6, 12
now creating features for lag:  1
merging with original dataframe
now creating features for lag:  2
merging with original dataframe
now creating features for lag:  3
merging with original dataframe
now creating features for lag:  6
merging with original dataframe
now creating features for lag:  12
merging with original dataframe
adding mean encoded feature for date_item_categ

In [82]:
train_df.shape

(10913850, 38)

In [86]:
# save intermediate output
train_df.to_pickle(path.join(DATA_PATH, 'train_df_temp.pkl'))

In [97]:
train_df = pd.read_pickle(path.join(DATA_PATH, 'train_df_temp.pkl'))

In [98]:
train_df.shape

(10913850, 38)

# Trend features 

For trend features, we have to once again start from the initial sales_train_df, since matirx is already a month-wise aggregated dataframe - this does not tell us the actual price of item at the time of sale.

In [99]:
# average item price across entire data
avg_item_price = sales_train_df.groupby(['item_id']).agg({'item_price': 'mean'})
avg_item_price.columns=['item_avg_price']
avg_item_price.reset_index(inplace=True)

# average item price monthwise
avg_date_block_item_price = sales_train_df.groupby(['date_block_num', 'item_id']).agg({'item_price': 'mean'})
avg_date_block_item_price.columns=['date_item_avg_price']
avg_date_block_item_price.reset_index(inplace=True)

# merge the above to original trainig dataframe

train_df = train_df.merge(avg_item_price, on='item_id', how='left')
train_df = train_df.merge(avg_date_block_item_price, on=['item_id', 'date_block_num'], how='left')

# for monthwise item price - create lagged features
lags = [1,2,3,4,5,6]
train_df = create_lagged_features(train_df, lags=lags, col='date_item_avg_price' )

now creating features for lag:  1
merging with original dataframe
now creating features for lag:  2
merging with original dataframe
now creating features for lag:  3
merging with original dataframe
now creating features for lag:  4
merging with original dataframe
now creating features for lag:  5
merging with original dataframe
now creating features for lag:  6
merging with original dataframe


In [100]:
train_df[train_df.date_block_num==1].head(30)

,date_block_num,shop_id,item_id,item_cnt_month,city_id,item_category_id,item_type_id,item_sub_type_id,date_avg_item_cnt,date_avg_item_cnt_lagged_1,...,date_item_sub_type_avg_item_cnt,date_item_sub_type_avg_item_cnt_lagged_1,item_avg_price,date_item_avg_price,date_item_avg_price_lagged_1,date_item_avg_price_lagged_2,date_item_avg_price_lagged_3,date_item_avg_price_lagged_4,date_item_avg_price_lagged_5,date_item_avg_price_lagged_6
365175,1,0,12,0.0,29,55,13,2,0.32441,NaN,...,0.188641,NaN,189.000000,189.000000,NaN,NaN,NaN,NaN,NaN,NaN
365176,1,0,27,0.0,29,19,5,10,0.32441,0.347176,...,0.827042,0.814994,1461.228571,1998.000000,2325.000000,NaN,NaN,NaN,NaN,NaN
365177,1,0,28,0.0,29,30,8,55,0.32441,0.347176,...,0.994397,1.126854,310.010465,416.875000,549.000000,NaN,NaN,NaN,NaN,NaN
365178,1,0,30,20.0,29,40,11,4,0.32441,NaN,...,0.266213,NaN,323.679206,383.921877,NaN,NaN,NaN,NaN,NaN,NaN
365179,1,0,31,11.0,29,37,11,1,0.32441,NaN,...,0.202230,NaN,578.630005,666.991044,NaN,NaN,NaN,NaN,NaN,NaN
365180,1,0,32,10.0,29,40,11,4,0.32441,0.347176,...,0.266213,0.293846,249.629240,337.771930,338.110349,NaN,NaN,NaN,NaN,NaN
365181,1,0,33,3.0,29,37,11,1,0.32441,0.347176,...,0.202230,0.209898,252.831928,484.170732,488.517241,NaN,NaN,NaN,NaN,NaN
365182,1,0,34,0.0,29,40,11,4,0.32441,0.347176,...,0.266213,0.293846,144.213115,149.000000,140.888889,NaN,NaN,NaN,NaN,NaN
365183,1,0,35,14.0,29,40,11,4,0.32441,0.347176,...,0.266213,0.293846,375.828056,367.396040,390.709091,NaN,NaN,NaN,NaN,NaN
365184,1,0,36,1.0,29,37,11,1,0.32441,NaN,...,0.202230,NaN,183.012195,525.000000,NaN,NaN,NaN,NaN,NaN,NaN


In order to extract what can be termed as price trend, we check for the most recent price available of the item prior to the current month, and see the relative difference.

In [ ]:
for lag in lags:
    train_df['perc_delta_price_lagged_' + str(lag)] = ((train_df['date_item_avg_price_lagged_' + str(lag)] 
                                                         - train_df['item_avg_price']) / train_df['item_avg_price'])

def select_trend_value(row):
    for lag in lags:
        if row['perc_delta_price_lagged_' + str(lag)]:
            return row['perc_delta_price_lagged_' + str(lag)]
    return 0
train_df['perc_detla_price_lagged'] = train_df.apply(select_trend_value, axis=1)
# fill missing values with zero
train_df['perc_detla_price_lagged'].fillna(0, inplace=True)

In [105]:
features_to_drop = (['date_item_avg_price_lagged_' + str(lag) for lag in lags]
                    + ['perc_delta_price_lagged_' + str(lag) for lag in lags] 
                    + ['item_avg_price'])
train_df = train_df.drop(columns=features_to_drop)

In [106]:
train_df.to_pickle(path.join(DATA_PATH, 'train_df_temp_2.pkl'))

In [145]:
train_df = pd.read_pickle(path.join(DATA_PATH, 'train_df_temp_2.pkl'))

In [146]:
train_df.shape

(10913850, 40)

## shop revenue trend

In this feature, we see the revenue of the shop compared relative to the average revenue of the shops (as per the training data). Since this information is not available for the current month, this has to be lagged by 1 month, since that is when you have the information available latest.

In [147]:
shop_revenue_df = sales_train_df.groupby(['date_block_num',
                                          'shop_id']).agg({'revenue': 'sum'})
shop_revenue_df.columns = ['date_shop_total_revenue']
shop_revenue_df.reset_index(inplace=True)
# shop average revenue across all months
shop_revenue_df['shop_avg_revenue'] = shop_revenue_df.groupby(['shop_id'])['date_shop_total_revenue'].transform('mean')


In [148]:
shop_revenue_df['delta_revenue'] = (shop_revenue_df['date_shop_total_revenue'] - shop_revenue_df['shop_avg_revenue']) / shop_revenue_df['shop_avg_revenue']


In [ ]:
train_df = train_df.merge(shop_revenue_df, on=['date_block_num', 'shop_id'], how='left')

In [150]:
train_df = create_lagged_features(train_df, [1], 'delta_revenue')
train_df = train_df.drop(columns=['delta_revenue', 'date_shop_total_revenue', 'shop_avg_revenue'])

now creating features for lag:  1
merging with original dataframe


In [151]:
train_df.shape

(10913850, 41)

# Months since last sale - for item-shop combination

### For item shop combination

In [127]:
shop_item_ordered_train_df = train_df.sort_values(by=['shop_id', 'item_id', 'date_block_num'])

In [131]:
shop_item_ordered_train_df['item_shop_last_sale_date'] = (shop_item_ordered_train_df
                                                          .groupby(['shop_id', 'item_id'])['date_block_num'].shift(1))
shop_item_ordered_train_df['mnths_since_last'] = shop_item_ordered_train_df['date_block_num'] - shop_item_ordered_train_df['item_shop_last_sale_date']
shop_item_ordered_train_df['mnths_since_last'] = shop_item_ordered_train_df['date_block_num'] - shop_item_ordered_train_df['item_shop_last_sale_date']
shop_item_ordered_train_df['mnths_since_last'].fillna(-1)

In [137]:
train_df = shop_item_ordered_train_df.sort_values(by=['date_block_num', 'shop_id', 'item_id'])

# Months since first sale - for shop item combination

In [138]:
train_df['item_shop_first_sale_month'] = train_df.groupby(['shop_id', 'item_id'])['date_block_num'].transform('min')
train_df['item_shop_mnths_since_first'] = train_df['date_block_num'] - train_df['item_shop_first_sale_month']

train_df['item_first_sale_month'] = train_df.groupby(['item_id'])['date_block_num'].transform('min')
train_df['item_mnths_since_first'] = train_df['date_block_num'] - train_df['item_first_sale_month']


train_df = train_df.drop(columns=['item_shop_first_sale_month', 'item_first_sale_month'])

In [141]:
train_df.to_pickle(path.join(DATA_PATH, 'train_df_temp_3.pkl'))

In [ ]:
train_df = pd.read_pickle(path.join(DATA_PATH, 'train_df_temp_3.pkl'))

# Final preparations

In [139]:
# month of the year
train_df['month'] = train_df['date_block_num'] % 12

In [156]:
# filter first 11 months?
train_df_final = train_df[train_df['date_block_num'] > 11]


In [159]:
# check na values
train_df_final.isnull().sum()

date_block_num                                         0
shop_id                                                0
item_id                                                0
item_cnt_month                                         0
city_id                                                0
item_category_id                                       0
item_type_id                                           0
item_sub_type_id                                       0
date_avg_item_cnt                                      0
date_avg_item_cnt_lagged_1                       1360816
date_item_avg_item_cnt                                 0
date_item_avg_item_cnt_lagged_1                  1360816
date_item_avg_item_cnt_lagged_2                  1548840
date_item_avg_item_cnt_lagged_3                  1744633
date_item_avg_item_cnt_lagged_6                  2300385
date_item_avg_item_cnt_lagged_12                 3405577
date_shop_avg_item_cnt                                 0
date_shop_avg_item_cnt_lagged_1

In [161]:
# Fill na values with zero for the 'item_cnt' columns having nulls

In [162]:
item_cnt_columns_with_null = [col for col in train_df_final.columns
                             if 'item_cnt' in col and train_df_final[col].isnull().any()]

In [164]:
for col in item_cnt_columns_with_null:
    train_df_final[col].fillna(0, inplace=True)

/opt/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [165]:
train_df_final.isnull().sum()

date_block_num                                         0
shop_id                                                0
item_id                                                0
item_cnt_month                                         0
city_id                                                0
item_category_id                                       0
item_type_id                                           0
item_sub_type_id                                       0
date_avg_item_cnt                                      0
date_avg_item_cnt_lagged_1                             0
date_item_avg_item_cnt                                 0
date_item_avg_item_cnt_lagged_1                        0
date_item_avg_item_cnt_lagged_2                        0
date_item_avg_item_cnt_lagged_3                        0
date_item_avg_item_cnt_lagged_6                        0
date_item_avg_item_cnt_lagged_12                       0
date_shop_avg_item_cnt                                 0
date_shop_avg_item_cnt_lagged_1